In [8]:
# import packages
import pandas as pd
import numpy as np
import os

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append("../../src/")


import oge.load_data as load_data
from oge.filepaths import *

year = 2022

# About
The purpose of this notebook is to update the `reference_tables/steam_units_to_remove.csv` table.
This table identifies units in CEMS that do not exist in the EIA data, and which only report steam data.

In [2]:
cems = load_data.load_cems_data(2022)
cems

,plant_id_eia,emissions_unit_id_epa,datetime_utc,operating_time_hours,gross_generation_mwh,steam_load_1000_lb,fuel_consumed_mmbtu,co2_mass_lb,nox_mass_lb,so2_mass_lb,plant_id_epa,co2_mass_measurement_code,nox_mass_measurement_code,so2_mass_measurement_code
0,6288,2,2022-01-01 09:00:00+00:00,1.0,52.0,0.0,NaN,NaN,NaN,NaN,6288,<NA>,<NA>,<NA>
1,6288,2,2022-01-01 10:00:00+00:00,1.0,52.0,0.0,NaN,NaN,NaN,NaN,6288,<NA>,<NA>,<NA>
2,6288,2,2022-01-01 11:00:00+00:00,1.0,52.0,0.0,NaN,NaN,NaN,NaN,6288,<NA>,<NA>,<NA>
3,6288,2,2022-01-01 12:00:00+00:00,1.0,52.0,0.0,NaN,NaN,NaN,NaN,6288,<NA>,<NA>,<NA>
4,6288,2,2022-01-01 13:00:00+00:00,1.0,52.0,0.0,NaN,NaN,NaN,NaN,6288,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35142667,57703,CT03,2023-01-01 02:00:00+00:00,0.0,NaN,0.0,NaN,NaN,NaN,NaN,57703,<NA>,<NA>,<NA>
35142668,57703,CT03,2023-01-01 03:00:00+00:00,0.0,NaN,0.0,NaN,NaN,NaN,NaN,57703,<NA>,<NA>,<NA>
35142669,57703,CT03,2023-01-01 04:00:00+00:00,0.0,NaN,0.0,NaN,NaN,NaN,NaN,57703,<NA>,<NA>,<NA>
35142670,57703,CT03,2023-01-01 05:00:00+00:00,0.0,NaN,0.0,NaN,NaN,NaN,NaN,57703,<NA>,<NA>,<NA>


In [24]:
cems_units = cems.groupby(["plant_id_epa", "plant_id_eia", "emissions_unit_id_epa"])[
    ["gross_generation_mwh", "steam_load_1000_lb","co2_mass_lb"]
].sum().reset_index()
cems_units = cems_units.merge(generator_unit_map, how="left", on=["plant_id_eia","emissions_unit_id_epa"])

In [29]:
cems_units[cems_units["plant_id_eia"] == 55212]

,plant_id_epa,plant_id_eia,emissions_unit_id_epa,gross_generation_mwh,steam_load_1000_lb,co2_mass_lb,generator_id
3671,55212,55212,1,866882.0,0.0,1.131372e+09,U1
3672,55212,55212,2,915101.0,0.0,1.174157e+09,U2


In [26]:
cems_units[cems_units["generator_id"].isna() & (cems_units["gross_generation_mwh"] == 0)]

,plant_id_epa,plant_id_eia,emissions_unit_id_epa,gross_generation_mwh,steam_load_1000_lb,co2_mass_lb,generator_id
721,1594,1594,11,0.0,111048.0,0.000000e+00,NaN
722,1594,1594,12,0.0,125561.0,0.000000e+00,NaN
752,1702,1702,A,0.0,19443.0,2.688114e+06,NaN
753,1702,1702,B,0.0,44874.0,5.821982e+06,NaN
1090,2503,2503,BLR114,0.0,980786.0,0.000000e+00,NaN
...,...,...,...,...,...,...,...
5404,880109,880109,B001,0.0,0.0,0.000000e+00,NaN
5405,880110,880110,1,0.0,63168.0,0.000000e+00,NaN
5406,880110,880110,2,0.0,72578.0,0.000000e+00,NaN
5407,880110,880110,3,0.0,98377.0,0.000000e+00,NaN


In [16]:
crosswalk = load_data.load_pudl_table("epacamd_eia")

In [28]:
crosswalk[crosswalk["plant_id_eia"] == 55212]

,report_year,plant_id_epa,emissions_unit_id_epa,generator_id_epa,plant_id_eia,boiler_id,generator_id
4851,2018,55212,1,U1,55212,U1,U1
4852,2018,55212,2,U2,55212,U2,U2
11188,2021,55212,1,U1,55212,U1,U1
11189,2021,55212,2,U2,55212,U2,U2


In [9]:
generator_unit_map = pd.read_csv(
        outputs_folder(f"{year}/subplant_crosswalk_{year}.csv"),
    )[["plant_id_eia", "generator_id", "emissions_unit_id_epa"]]

In [10]:
generator_unit_map

,plant_id_eia,generator_id,emissions_unit_id_epa
0,1,5.1,NaN
1,1,3,NaN
2,1,1,NaN
3,1,2,NaN
4,1,5,NaN
...,...,...,...
33627,880109,NaN,B001
33628,880110,NaN,1
33629,880110,NaN,2
33630,880110,NaN,3


In [ ]:
eia_